In [ ]:
### Import Libraries
import tensorflow as tf
import os
#import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
# Importing DenseNet121
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

## defining variables

In [ ]:
RANDOM_STATE = 2022
tf.random.set_seed(RANDOM_STATE)

EPOCHS = 30 

BATCH_SIZE = 32

TEST_SIZE = 0.2

IMG_HEIGHT = 128

IMG_WIDTH = 128

# Set parameters for decoration of plots
params = {'legend.fontsize' : 'large',
          'figure.figsize'  : (15,10),
          'axes.labelsize'  : 'x-large',
          'axes.titlesize'  :'x-large',
          'xtick.labelsize' :'large',
          'ytick.labelsize' :'large',
         }

CMAP = plt.cm.brg

plt.rcParams.update(params)

In [ ]:
data_dir = r"/content/drive/MyDrive/01_DATASET"  # dataset location

Spliting the Data

In [ ]:
# load data and split in training and validation from a sub dir

# training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=TEST_SIZE,
    subset="training",
    seed=RANDOM_STATE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)

# testing dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=TEST_SIZE,
    subset="validation",
    seed=RANDOM_STATE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)

Found 412 files belonging to 2 classes.
Using 330 files for training.
Found 412 files belonging to 2 classes.
Using 82 files for validation.


In [ ]:
## checking the classes
class_names = train_ds.class_names

print('Total: {:3d} Classes; namely : {:s}'.format(len(class_names), str(class_names)))

Total:   2 Classes; namely : ['not receipt', 'receipt']


In [ ]:
## Optimize for performance
#loading data into cache for faster execution
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

### MOdel preprocessing

In [ ]:
# calling the densnet model
DenseNet = DenseNet121(weights='imagenet',include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)) 

In [ ]:
for layer in DenseNet.layers:
    layer.trainable = False

In [ ]:
DenseNet.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_6 (ZeroPadding2  (None, 134, 134, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d_6[0][0]']       
                                                                                        

In [ ]:
## rescaling and resizing
resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255)
])

# ### data augmentation
# data_augmentation = tf.keras.Sequential(
#   [
#     tf.keras.layers.RandomFlip("horizontal",
#                       input_shape=(IMG_HEIGHT,
#                                   IMG_WIDTH,
#                                   3)),
#     tf.keras.layers.RandomRotation((-0.1,0.1)),
#     tf.keras.layers.RandomZoom(-0.1,1),
#   ]
# )

In [ ]:
# for image, _ in train_ds.take(1):
#   plt.figure(figsize=(10, 10))
#   first_image = image[0]
#   for i in range(9):
#     # ax = plt.subplot(3, 3, i + 1)
#     augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
#     plt.imshow(augmented_image[0] / 255)
#     plt.axis('off')

In [ ]:
num_classes = len(class_names)

## Model building

In [ ]:
# Create Model objecct
def build_model(d_model, n_classes):

    krnl_init = tf.keras.initializers.GlorotUniform(
                                       seed=RANDOM_STATE )


    model = tf.keras.Sequential([resize_and_rescale, data_augmentation])

    model.add(DenseNet)

    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Flatten())

    # First Dense set
    model.add(tf.keras.layers.Dense(1024,
                                  kernel_initializer = krnl_init,
                                  use_bias = False))

    model.add(tf.keras.layers.ReLU())

    model.add(tf.keras.layers.Dropout(0.5))   

    # output layer
    model.add(tf.keras.layers.Dense(n_classes,
                                  kernel_initializer = krnl_init
                                  ))

    return model

In [ ]:
model = build_model(DenseNet,num_classes) 

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9,
                              patience=5, min_lr=0.001)

history = model.fit(train_ds,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data=test_ds, callbacks=[reduce_lr])

Epoch 1/30
11/11 [==============================] - 37s 2s/step - loss: 65.7741 - accuracy: 0.6485 - val_loss: 14.6767 - val_accuracy: 0.8780 - lr: 0.0100
Epoch 2/30
11/11 [==============================] - 25s 2s/step - loss: 7.5599 - accuracy: 0.8697 - val_loss: 8.7317 - val_accuracy: 0.9146 - lr: 0.0100
Epoch 3/30
11/11 [==============================] - 23s 2s/step - loss: 5.4351 - accuracy: 0.8939 - val_loss: 3.4806 - val_accuracy: 0.9512 - lr: 0.0100
Epoch 4/30
11/11 [==============================] - 27s 3s/step - loss: 2.9422 - accuracy: 0.9273 - val_loss: 2.5041 - val_accuracy: 0.9634 - lr: 0.0100
Epoch 5/30
11/11 [==============================] - 23s 2s/step - loss: 2.1167 - accuracy: 0.9182 - val_loss: 2.1466 - val_accuracy: 0.9512 - lr: 0.0100
Epoch 6/30
11/11 [==============================] - 24s 2s/step - loss: 2.3732 - accuracy: 0.9152 - val_loss: 1.6282 - val_accuracy: 0.9634 - lr: 0.0100
Epoch 7/30
11/11 [==============================] - 26s 2s/step - loss: 0.6424 -

In [ ]:
final_df = pd.DataFrame(history.history)
final_df

,loss,accuracy,val_loss,val_accuracy,lr
0,65.774101,0.648485,14.676737,0.878049,0.01000
1,7.559856,0.869697,8.731734,0.914634,0.01000
2,5.435139,0.893939,3.480642,0.951219,0.01000
3,2.942191,0.927273,2.504066,0.963415,0.01000
4,2.116694,0.918182,2.146554,0.951219,0.01000
5,2.373204,0.915152,1.628174,0.963415,0.01000
6,0.642417,0.936364,1.270962,0.951219,0.01000
7,0.770491,0.945455,1.561911,0.951219,0.01000
8,0.865774,0.954545,1.037029,0.963415,0.01000
9,0.919977,0.933333,0.874507,0.963415,0.01000


## Saving the weights

In [ ]:
model.save(r'/content/drive/MyDrive/02_weights/best_model.h5')